## Authors: Pablo Mollá, Pavlo Poliuha and Junjie Chen

# Deep Learning - Laboratory Exercise 2

- **WARNING:** you must have finished the previous exercise before this one as you will re-use parts of the code.

- In the $\textcolor{green}{\text{first laboratory exercise}}$, we built a $\textcolor{green}{\text{simple linear classifier}}$. Although it can give reasonable results on the MNIST dataset (~92.5% of accuracy), deeper neural networks can achieve more the 99% accuracy. However, it can quickly become really impracical to explicitly code forward and backward passes. Hence, it is useful to rely on an auto-diff library where we specify the forward pass once, and the backward pass is automatically deduced from the computational graph structure.

In $\textcolor{red}{\text{this laboratory exercise}}$, we will build a $\textcolor{red}{\text{small and simple auto-diff library}}$ that mimics the autograd mechanism from Pytorch (of course, we will simplify a lot!)


In [74]:
def load_mnist(fname="mnist.pkl.gz"):
    f = gzip.open(fname, 'rb')

    if(sys.version_info.major==2):
        train_set, valid_set, test_set = pickle.load(f) # compatibility issue between python 2.7 and 3.4
    else:
        train_set, valid_set, test_set = pickle.load(f, encoding='latin-1') # compatibility issue between python 2.7 and 3.4
    f.close()

    # Shuffle
    train_X = train_set[0]
    train_y = train_set[1]
    valid_X = valid_set[0]
    valid_y = valid_set[1]

    train_perm = np.random.permutation(train_X.shape[0])
    train_set = [train_X[train_perm,:],train_y[train_perm]]

    valid_perm = np.random.permutation(valid_X.shape[0])
    valid_set = [valid_X[valid_perm,:],valid_y[valid_perm]]

    return train_set, valid_set, test_set

In [75]:
# import libs that we will use
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import gzip
import pickle
import dataset_loader
# To load the data we will use the script of Gaetan Marceau Caron
# You can download it from the course webiste and move it to the same directory that contains this ipynb file
#!pip install dataset_loader
%matplotlib inline

In [76]:
# Download mnist dataset 
if("mnist.pkl.gz" not in os.listdir(".")):
    # this link doesn't work any more,
    # seach on google for the file "mnist.pkl.gz"
    # and download it
    !wget http://deeplearning.net/data/mnist/mnist.pkl.gz

# if you have it somewhere else, you can comment the lines above
# and overwrite the path below
mnist_path = "./mnist.pkl.gz"

In [77]:
# Load the 3 splits
train_data, dev_data, test_data = load_mnist(mnist_path)

## Computation Graph

Instead of directly manipulating numpy arrays, we will manipulate abstraction that contains:
- a value (i.e. a numpy array)
- a bool indicating if we wish to compute the gradient with respect to the value
- the gradient with respect to the value
- the operation to call during backpropagation

There will be two kind of nodes:
- ComputationGraphNode: a generic computation node
- Parameter: a computation node that is used to store parameters of the network. Parameters are always leaf nodes, i.e. they cannot be build from other computation nodes.

Our implementation of the backward pass will be really simple and incorrect in the general case (i.e. won't work with computation graph with loops).
We will just apply the derivative function for a given tensor and then call the ones of its antecedents, recursively.
This simple algorithm is good enough for this exercise.

Note that a real implementation of backprop will store temporary values during forward that can be used during backward to improve computation speed. We do not do that here.


In [78]:
class ComputationGraphNode(object):
    
    def __init__(self, data, require_grad=False):
        # we initialise the value of the node and the grad
        if(not isinstance(data, np.ndarray)):
            data = np.array(data)
        self.value = data
        self.grad = None
        
        self.require_grad = require_grad
        self.func = None
        self.input_nodes = None
        self.func_parameters = []
    
    def set_input_nodes(self, *nodes):
        self.input_nodes = list(nodes)

    def set_func_parameters(self, *func_parameters):
        self.func_parameters = list(func_parameters)
    
    def set_func(self, func):
        self.func = func

    def zero_grad(self):
        if self.grad is not None:
            self.grad.fill(0)

    def set_gradient(self, gradient):
        """
        Accumulate gradient for this tensor
        """
        if gradient.shape != self.value.shape:
            print(gradient.shape, self.value.shape)
            raise RuntimeError("Invalid gradient dimension")
        if self.grad is None:
            self.grad = gradient
        else:
            self.grad += gradient
    
    def backward(self, g=None):
        if g is None:
            g = self.value.copy()
            g.fill(1.)
        self.set_gradient(g)
        if self.func is not None:
            grad_list = self.func.backward(*(self.input_nodes + self.func_parameters + [g]))
            for input_node, ngrad in zip(self.input_nodes, grad_list):
                input_node.backward(ngrad)
    
    def __add__(self, y):
        if not isinstance(y, ComputationGraphNode):
            y = ComputationGraphNode(y)
        return Addition()(self, y)

    def __getitem__(self, slice):
        return Selection()(self, slice)

    def __str__(self):
        return self.value.__str__()

    def __repr__(self):
        return self.value.__str__()

class Parameter(ComputationGraphNode):
    def __init__(self, data, name="default"):
        super().__init__(data, require_grad=True)
        self.name  = name

    def backward(self, g=None):
        if g is not None:
            self.set_gradient(g)

The class `Operation` is a class that three methods you should reimplement only the forward and the backward methods.
* The `forward` method compute the function w.r.t inputs and return a new node that must contains information for backward pass.
* The `backward` functions compute the gradient of the function w.r.t gradient of the output and other informations (forward pass input, parameter of the function...).**It should return a tuple**

For better understanding below two operation are implemented, the selection and the addition (notice that it should not works yet since we do not defined what is a node)

In [79]:
class Operation(object):
    @staticmethod
    def forward(*args):
        raise NotImplementedError("It is an abstract method")
    
    def __call__(self, *args):
        output_node = self.forward(*args)
        output_node.set_func(self)
        return output_node
        
    @staticmethod
    def backward(*args):
        pass
class Addition(Operation):
    @staticmethod
    def forward(x, y):
        output_array = x.value + y.value
        output_node = ComputationGraphNode(output_array)
        output_node.set_input_nodes(x, y)
        return output_node

    @staticmethod
    def backward(x, y, gradient):
        return (gradient, gradient)

class Selection(Operation):
    @staticmethod
    def forward(x, slice):
        np_x = x.value

        output_array = np_x.__getitem__(slice)
        
        output_node = ComputationGraphNode(output_array)
        output_node.set_input_nodes(x)
        output_node.set_func_parameters(slice)

        return output_node
        
    @staticmethod
    def backward(x, slice, gradient):
        np_x = x.value

        cgrad = np_x.copy()
        cgrad.fill(0)
        cgrad.__setitem__(slice, gradient)
        
        return cgrad,

**Question 1** Complete the following class 

### ReLU (Rectified Linear Unit)

The ReLU (Rectified Linear Unit) function is a piecewise linear function that outputs the input directly if it is positive, and outputs zero otherwise. It is defined mathematically as:

$$ ReLU(x) = \max(0, x) $$

#### Analytical Explanation of the Forward Pass

The forward pass of the ReLU operation involves applying the ReLU function to each component of the input vector $x$. For each element $x_i$ of $x$, the ReLU function computes:

$$ ReLU(x_i) = 
\begin{cases} 
x_i & \text{if } x_i > 0 \\
0 & \text{if } x_i \leq 0
\end{cases}
$$

- This operation is vectorized across the entire input vector $x$, resulting in an output vector where each element is the result of applying the ReLU function to the corresponding element of $x$. The purpose of applying ReLU is to introduce non-linearity into the model, allowing it to learn more complex patterns.

#### Analytical Explanation of the Backward Pass

During the backward pass, the gradient of the loss function with respect to the input of the ReLU operation needs to be calculated to propagate the error through the network. This gradient, denoted $\frac{\partial \mathcal{L}}{\partial x_i}$ for each input component $x_i$, depends on the derivative of the ReLU function.

- The derivative of the ReLU function with respect to its input is:

$$
\frac{d ReLU}{d x_i} = 
\begin{cases} 
1 & \text{if } x_i > 0 \\
0 & \text{if } x_i \leq 0
\end{cases}
$$

Thus, during the backward pass, for each component $x_i$ of $x$, the gradient of the loss $\mathcal{L}$ with respect to $x_i$ is modified according to the derivative of ReLU. If $x_i$ was $\textcolor{green}{\text{positive}}$ (and thus passed through ReLU unchanged), the gradient $\frac{\partial \mathcal{L}}{\partial x_i}$ is passed backward unchanged. If $x_i$ was $\textcolor{red}{\text{non-positive}}$ (and ReLU output zero for it), the gradient with respect to $x_i$ is "zeroed" out because changes to $x_i$ would not affect the loss $\mathcal{L}$ due to the flat (zero gradient) region of ReLU for $x_i \leq 0$.

- The $\textcolor{orange}{\text{overall operation can be represented as element-wise multiplication}}$ of the incoming gradient (from the subsequent layer or operation) by the gradient of ReLU with respect to its input. This gradient propagation respects the chain rule in calculus, which states that to compute the gradient of a composite function, you multiply the gradients of the composed functions.

- In practical terms, this means for a given gradient vector $\nabla \mathcal{L}$ with respect to the output of ReLU, the backward pass computes:

$$
\nabla \mathcal{L} \odot \frac{d ReLU}{d x} = 
\begin{cases} 
\nabla \mathcal{L}_i & \text{if } x_i > 0 \\
0 & \text{if } x_i \leq 0
\end{cases}
$$

Where $\odot$ denotes element-wise multiplication, and $\nabla \mathcal{L}_i$ is the $i-th$ component of the gradient vector $\nabla \mathcal{L}$.

This process ensures that the error signal is properly propagated backward through the network, allowing for the optimization of the model's weights during training via gradient descent or related algorithms.

In [80]:
class ReLU(Operation):
    @staticmethod
    def forward(x):
        # A copy of the input node's value is created to 
        # ensure the original data remains unchanged
        np_x = x.value.copy()

        # All negative elements in this copy are set to zero. 
        # This operation implements the ReLU function, which 
        # outputs the input directly if it's positive and zero 
        # otherwise.
        np_x[np_x < 0] = 0

        # A new ComputationGraphNode is instantiated with 
        # the processed values as its value. This new node 
        # represents the output of the ReLU operation.
        output_node = ComputationGraphNode(np_x)

        # The new node's input nodes are set to the input 
        # node x, establishing a link in the computational graph.
        output_node.set_input_nodes(x)
        
        # Returns the output node, which is a ComputationGraphNode 
        # containing the result of applying the ReLU function to 
        # the input.
        return output_node
        
    @staticmethod
    def backward(x, gradient):
        # A copy of the input node's value is created to prevent 
        # modifying the original data.
        np_x = x.value.copy()

        # A mask is created where all elements of the input node's 
        # value that are less than zero are set to zero.
        np_x[np_x < 0] = 0

        # All elements of the mask that are greater than or 
        # equal to zero are set to one.
        np_x[np_x >= 0] = 1

        # Returns a tuple containing the product of the local 
        # gradients and the incoming gradient, which represents 
        # the gradient of the loss with respect to the input of 
        # the ReLU function.
        return np_x * gradient,

### Hyperbolic Tangent

The following code defines a $\textcolor{orange}{\text{class TanH}}$ that represents the hyperbolic tangent activation function, a commonly used activation function in neural networks that scales the input to be between -1 and 1. The class provides methods for computing the forward pass ($\textcolor{orange}{\text{forward}}$) and the derivative for the backward pass ($\textcolor{orange}{\text{backward}}$) of the activation function within a computational graph framework.


#### 1. TanHCompute Method

- **Purpose**: This static method computes the hyperbolic tangent of the input $x$. The computation is stabilized by subtracting the maximum absolute value of $x$ from $x$ to prevent numerical overflow in the exponentiation steps, especially for large positive or negative values of $x$.
  
- **Operation**: Given an input $x$, the method computes $\tanh(x)$ using the formula:
  
  $$
  \tanh(x) = \frac{e^{x - \max(|x|)} - e^{-x - \max(|x|)}}{e^{x - \max(|x|)} + e^{-x - \max(|x|)}}
  $$
  
  This formula is derived from the definition of $\tanh(x)$ but with a modification for numerical stability by incorporating $\max(|x|)$.

#### 2. Forward Method

- **Purpose**: Computes the forward pass of the TanH operation on a given input node $x$. This method applies the TanH activation function to each element of the input.

- **Operation**: 
  - A copy of the input node's value $(x.value)$ is made to ensure the original data remains unchanged.
  - The $TanHCompute$ method is called with this copy to apply the TanH activation function, resulting in an array where each input element is transformed by $\tanh(x)$.
  - A new $ComputationGraphNode$ is created with the transformed array as its value, representing the output of the TanH operation.
  - The new node's input nodes are set to the input node $x$, linking the current operation in the computational graph.
  
- **Output**: The method returns the output node that contains the result of applying the TanH function to the input.

#### 3. Backward Method

- **Purpose**: Computes the gradient of the loss function with respect to the input of the TanH operation during the backward pass of backpropagation.

- **Operation**: 
  - A copy of the input node's value is again made.
  
  - The $TanHCompute$ method is applied to this copy to get the TanH activated values.
  
  - The derivative of the TanH function with respect to its input is computed using the identity $$\frac{d \tanh(x)}{dx} = 1 - \tanh^2(x)$$
  
  This results in an array where each element is the local gradient of the TanH function at the corresponding input value.
  
  - The local gradients are then element-wise multiplied by the incoming gradient ($gradient$) from the subsequent operation or layer. This multiplication applies the chain rule, combining the local gradient with the gradient of the subsequent operations to get the gradient of the loss with respect to the TanH input.
  
- **Output**: The method returns a tuple containing the gradient of the loss with respect to the TanH input. This gradient is then used to update parameters earlier in the network during backpropagation.


In [81]:
class TanH(Operation):
    @staticmethod
    def TanHCompute(x):
        max_x = np.abs(x).max()
        return (np.exp(x-max_x) - np.exp(-x-max_x))/(np.exp(x-max_x) + np.exp(-x-max_x))
    
    @staticmethod
    def forward(x):
        np_x = x.value.copy()
        np_x = TanH.TanHCompute(np_x)
        output_node = ComputationGraphNode(np_x)
        output_node.set_input_nodes(x)
        return output_node
        
    @staticmethod
    def backward(x, gradient):
        np_x = x.value.copy()
        np_x = TanH.TanHCompute(np_x)
        np_x = 1 - np_x**2
        return np_x * gradient,
        

**Question 2:** Next, we implement the affine transform operation.
You can reuse the code from the third lab exercise, with one major difference: you have to compute the gradient with respect to x too!

#### 1. Affine Transformation

The $\textcolor{orange}{\text{class affine\_transform}}$ models an affine transformation operation as part of a computational graph. An affine transformation is a linear mapping method that preserves points, straight lines, and planes. In neural networks, this operation is often referred to as a fully connected layer or a dense layer, represented mathematically as $y = Wx + b$, where:

- W is the weight matrix

- x is the input vector

- b is the bias vector

- y is the output vector

#### 2. Forward Method

- **Purpose**: Computes the $\textcolor{orange}{\text{forward pass of the affine transformation}}$.

- **Operations**:

    - $npx = x.value$: Extracts the actual value of the input node $x$.

    - $npW = W.value$: Extracts the actual value of the weight matrix $W$.

    - $npb = b.value$: Extracts the actual value of the bias vector $b$.

    - $\text{output\_node = ComputationGraphNode(npW @ npx.T + npb)}$: Performs the affine transformation using matrix multiplication $@$ between $W$ and $x^T$ (transpose of $x$) and then adds the bias $b$. The result is wrapped in a $ComputationGraphNode$ to be used in further operations in the computational graph.

    - $\text{output\_node.set\_input\_nodes(W, b, x)}$: Sets $W$, $b$, and $x$ as the input nodes to the current $\text{output\_node}$. This linkage is crucial for backpropagation, as it allows the graph to trace which nodes contributed to the final output.

    - $\text{return output\_node}$: Returns the node containing the output of the affine transformation.

### 3. Backward Method

- **Purpose**: Computes the gradients of the loss function with respect to the affine transformation's parameters and inputs during the $\textcolor{orange}{\text{backward pass}}$.

- **Operations**:

    - $npx = x.value$: Retrieves the input vector $x$.

    - $npW = W.value$: Retrieves the weight matrix $W$.
    
    - $npb = b.value$: Retrieves the bias vector $b$, though it's not directly used in the gradient calculations below.
    
    - $npg = gradient$: The gradient of the loss with respect to the output of the affine transformation.
    
    - $\text{grad\_x = np.dot(npW.T, npg)}$: Calculates the gradient with respect to the input $x$ by dotting the transpose of $W$ with the gradient $npg$. This follows from the chain rule in calculus and reflects how changes in $x$ influence the loss through the affine transformation.
    
    - $\text{grad\_w = np.matmul(npg.reshape(-1,1), npx.reshape(1,-1))}$: Computes the gradient with respect to the weights $W$. This is done by taking the outer product of the gradient $npg$ with respect to the output and the input $x$, which indicates how the loss changes with changes in $W$.

    - $\text{grad\_b = npg}$: The gradient with respect to the bias $b$ is directly the gradient of the loss with respect to the output of the affine transformation, as each element of $b$ is added directly to each corresponding element of the output $y$, affecting the loss independently of the input $x$.
    
    - $\text{return grad\_w, grad\_b, grad\_x}$: Returns the gradients with respect to $W$, $b$, and $x$, which are used to update these parameters during the optimization process.


In [82]:
class affine_transform(Operation):
    @staticmethod
    def forward(W, b, x):
        npx = x.value
        npW = W.value
        npb = b.value
        output_node = ComputationGraphNode(npW @ npx.T + npb)
        output_node.set_input_nodes(W, b, x)
        return output_node
        
    @staticmethod
    def backward(W, b, x, gradient):
        npx = x.value
        npW = W.value
        npb = b.value
        npg = gradient
        grad_x = np.dot(npW.T, npg)
        grad_w = np.matmul(npg.reshape(-1,1),npx.reshape(1,-1))
        grad_b = npg
        return grad_w, grad_b, grad_x

**Question 3:** Define the NLL operation

We recall that:
 
- $$nll(x, y)= -log\left(\frac{e^{x_{y}}}{ \sum\limits_{i=1}^n e^{x_{ j}} }\right) = -x_{y} + log(\sum\limits_{i=1}^n e^{x_{ j} })$$

- $$
    \begin{align*}
        \frac{\partial nll(x, y)}{\partial x_i} &= - \mathbb{1}_{y = i} + \frac{\partial log(\sum\limits_{i=1}^n e^{x_{ j} })}{\partial\sum\limits_{i=1}^n e^{x_{ j} }}\frac{\sum\limits_{i=1}^n e^{x_{ j} }}{\partial x_i} \\
        &= - \mathbb{1}_{y = i} + \frac{e^{x_i}}{\sum\limits_{i=1}^n e^{x_{ j} }} 
    \end{align*}
$$

### Negative Log-Likelihood


The $\textcolor{orange}{\text{class nll}}$ that models the $\textcolor{green}{\text{Negative log-likelihood (NLL)}}$ loss operation, commonly used in classification tasks. It utilizes the $\textcolor{red}{\text{softmax function}}$ to convert logits into probabilities, which are then used to compute the NLL loss. The class provides methods for computing both the forward and backward passes of this operation within a computational graph.

#### 1. Softmax Method

- **Purpose**: Computes the $\textcolor{red}{\text{softmax}}$ of a vector $x$. The softmax function is used to normalize an input vector into a probability distribution of predicted classes.

- **Operations**:

  - $\text{e\_x = np.exp(x - np.max(x))}$: Computes the exponential of each element in the input vector $x$, shifted by the maximum element in $x$ for numerical stability. Subtracting the max value prevents potential overflow by ensuring that the largest exponent argument is 0.

  - $\text{return e\_x / e\_x.sum()}$: Normalizes these exponentials to make their sum equal to 1, effectively converting the logits into probabilities.

#### 2. Forward Method

- **Purpose**: Computes the $\textcolor{orange}{\text{forward pass}}$ of the NLL loss for a predicted probability distribution and a true class index $y$.

- **Operations**:
  - $\text{np\_x = x.value.copy()}$: Creates a copy of the input vector $x$, which represents the logits before applying softmax.

  - $\text{np\_x = nll.softmax(np\_x)}$: Applies the $\textcolor{red}{\text{softmax}}$ method to the logits to obtain the predicted probabilities.

  - $\text{output\_node = ComputationGraphNode(-np.log(np\_x[y]))}$: Computes the $\textcolor{green}{\text{negative logarithm}}$ of the probability associated with the true class $y$. This quantity represents the NLL loss for the given prediction and true class. 
  
  The result is wrapped in a $ComputationGraphNode$ to be used in further operations or for loss backpropagation.
  
  - $\text{output\_node.set\_input\_nodes(x)}$: Links the input logits node $x$ as an input to the current operation in the computational graph.

  - $\text{output\_node.set\_func\_parameters(y)}$: Stores the true class index $y$ as a parameter of the operation for potential reference in the backward pass or other purposes.
  
#### 3. Backward Method

- **Purpose**: Computes the gradient of the NLL loss with respect to the input logits $x$ during the $\textcolor{orange}{\text{backward pass}}$.

- **Operations**:
  - $\text{np\_x = x.value.copy()}$: Retrieves the input logits.
  
  - $\text{g\_x = nll.softmax(np\_x)}$: Applies the $\textcolor{red}{\text{softmax}}$ to the logits to obtain the gradient of the softmax operation, which in this context represents the probabilities after applying softmax.

  - $g\_x[y] -= 1$: Adjusts the element of the softmax output corresponding to the true class $y$ by subtracting 1. This operation is based on the derivative of the $\textcolor{green}{\text{NLL}}$ loss with respect to the input logits, which simplifies to $p_i - 1$ for the true class $i=y$ and remains as $p_j$ for all other classes $j \neq y$, where $p$ are the softmax probabilities.

  - $\text{return g\_x * gradient,}$: Scales the computed gradient by the incoming gradient ($gradient$). In most cases, this incoming gradient will be 1, as the $\textcolor{green}{\text{NLL}}$ loss is typically the final operation whose gradient initiates the backward pass. The result is a vector of gradients with respect to each logit, which is used to update the weights that contributed to $x$ during optimization.


In [83]:
class nll(Operation):
    @staticmethod
    def softmax(x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()
    @staticmethod
    def forward(x, y):
        np_x = x.value.copy()
        np_x = nll.softmax(np_x)
        output_node = ComputationGraphNode(-np.log(np_x[y]))
        output_node.set_input_nodes(x)
        output_node.set_func_parameters(y)
        return output_node
        
    @staticmethod
    def backward(x, y, gradient=1):
        np_x = x.value.copy()
        g_x = nll.softmax(np_x)
        g_x[y] -= 1
        return g_x * gradient, 

# Module

Neural networks or parts of neural networks will be stored in Modules.
They implement method to retrieve all parameters of the network and subnetwork.

In [84]:
class Module:
    def __init__(self):
        raise NotImplemented("")
        
    def parameters(self):
        ret = []
        for name in dir(self):
            o = self.__getattribute__(name)

            if type(o) is Parameter:
                ret.append(o)
            if isinstance(o, Module) or isinstance(o, ModuleList):
                ret.extend(o.parameters())
        return ret

# if you want to store a list of Parameters or Module,
# you must store them in a ModuleList instead of a python list,
# in order to collect the parameters correctly
class ModuleList(list):
    def parameters(self):
        ret = []
        for m in self:
            if type(m) is Parameter:
                ret.append(m)
            elif isinstance(m, Module) or isinstance(m, ModuleList):
                ret.extend(m.parameters())
        return ret

# Initialization and optimization

**Question 1:** Implement the different initialisation methods

#### Weights

- `Xavier initialization`, also known as Glorot initialization, is a method of initializing the weights of a neural network to help with the convergence during training. It is named after Xavier Glorot, who, along with `Yoshua Bengio`, introduced the method in a paper in 2010 to address the problems of training deep neural networks.

- The `key` idea behind Xavier initialization is to initialize the weights in such a way that the variance of the outputs of each layer is the same as the variance of its inputs. This helps prevent the gradients from becoming too small or too large, a problem that can lead to either vanishing gradients or exploding gradients, respectively.

- The Xavier initialization sets a layer's weights $W$ using a random distribution that has a zero mean and a specific variance that depends on the number of input and output units of the layer:

$$ \text{Var}(W) = \frac{2}{n_{\text{in}} + n_{\text{out}}}$$

Where:
- $n_{\text{in}}$ is the number of units (neurons) in the layer's input (i.e., the size of the previous layer).
- $n_{\text{out}}$ is the number of units in the layer's output (i.e., the size of the current layer).

1. For the `uniform distribution`, the weights are initialized using the following range:

$$ W \sim \text{Uniform}\left(-\sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}}, \sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}}\right) $$

- We will use this one for our Neural Network.

2. For the `normal distribution`, they are initialized using:

$$ W \sim \mathcal{N}\left(0, \sqrt{\frac{2}{n_{\text{in}} + n_{\text{out}}}}\right) $$

In practice, when using libraries like TensorFlow or PyTorch, Xavier initialization can often be specified by using the appropriate initializer when constructing layers of a neural network. For example with TensorFlow, we could have used `tf.keras.initializers.GlorotUniform()` for uniform Xavier initialization or `tf.keras.initializers.GlorotNormal()` for normal Xavier initialization.

#### Bias

The Bias vector is set to 0 due to 2 main reasons:

- `Kickstarting` Learning: We can think of the bias as a starting point for the neurons. Setting the bias to zero is like starting from a neutral position, allowing the weights (which are initialized using the Glorot method) to take the lead in learning from the data. This way, each neuron begins learning without any initial push in a particular direction, making it easier for the network to adjust and learn the patterns in the data effectively.

- `Keeping Things Balanced`: By starting the biases at zero, we help keep the outputs of neurons initially centered around zero. This balance is crucial because it helps avoid extreme values in the neurons' outputs at the beginning of training, making it smoother for the network to learn and adjust as training progresses.

In [85]:
def zero_init(b):
    b[:] = 0.

def glorot_init(W):
    W[:, :] = np.random.uniform(-np.sqrt(6. / (W.shape[0] + W.shape[1])), np.sqrt(6. / (W.shape[0] + W.shape[1])), W.shape)
    
# Look at slides for the formula!
def kaiming_init(W):
    W[:, :] = np.random.uniform(-np.sqrt(6. / W.shape[0]), np.sqrt(6. / W.shape[0]), W.shape)

**Question 2:** Implement the SGD 

We will implement the Stochastic gradient descent through an object, in the init function this object will store the different parameters (in a list format). The step function will update the parameters (see slides), notice that the gradient is stored in the nodes (grad attribute). Finally it will be necessary after each update to reset all the gradient to zero (in the method zero_grad) because we do not want to accumumlate gradient of all previous step.

In [86]:
# Simple gradient descent optimizer
class SGD:
    def __init__(self, params, lr=0.1):
        self.params = params
        self.lr = lr
        
    def step(self):
        for p in self.params:
            p.value[:] = p.value - self.lr * p.grad
        
    def zero_grad(self):
        for p in self.params:
            p.zero_grad()

# Networks and training loop

We first create a simple linear classifier, similar to the first lab exercise.

In [87]:
class LinearNetwork(Module):
    def __init__(self, dim_input, dim_output):
        # build the parameters
        self.W = Parameter(np.ndarray((dim_output, dim_input)), name="W")
        self.b = Parameter(np.ndarray((dim_output,)), name="b")
        self.init_parameters()
        
    def init_parameters(self):
        # init parameters of the network (i.e W and b)
        glorot_init(self.W.value)
        zero_init(self.b.value)
        
    def forward(self, x):
        return  affine_transform()(self.W, self.b, x)

In [88]:
np.random.seed(42)

In [89]:
# Those lines should be executed correctly
lin1 = LinearNetwork(784, 10)
lin2 = LinearNetwork(10, 5)
x = ComputationGraphNode(train_data[0][0])
a = lin1.forward(x + x)
b = TanH()(a)
c = lin2.forward(b)
c.backward()
x.grad[0:10]

array([-0.18810699,  0.0176133 , -0.24328452, -0.37984876, -0.27928876,
       -0.09780842, -0.07722615, -0.21863881,  0.23739514, -0.23216239])

We will train several neural networks.
Therefore, we encapsulate the training loop in a function.

**warning**: you have to call optimizer.zero_grad() before each backward pass to reinitialize the gradient of the parameters!

In [90]:
def evaluate_accuracy(network, data):
    correct = 0
    total = len(data[0])
    for i in range(total):
        x = ComputationGraphNode(data[0][i])
        y = data[1][i]
        prediction = network.forward(x)
        if np.argmax(prediction.value) == y:
            correct += 1
    return correct / total

In [91]:
def training_loop(network, optimizer, train_data, dev_data, n_epochs=10):
    for epoch in range(n_epochs):
        # shuffle the data
        perm = np.random.permutation(train_data[0].shape[0])
        total_loss = []
        for i in perm:
            # forward pass
            x = ComputationGraphNode(train_data[0][i])
            y = train_data[1][i]
            a = network.forward(x)
            loss = nll()(a, y)
            total_loss.append(loss.value)
            # backward pass
            loss.backward()
            optimizer.step()
            # zero the gradients
            optimizer.zero_grad()
            
        print("Epoch", epoch, "Loss", np.mean(total_loss))
        dev_accuracy = evaluate_accuracy(network, dev_data)
        print(f"Epoch {epoch}, Dev Accuracy: {dev_accuracy * 100:.2f}%")

In [93]:
dim_input = 28*28
dim_output = 10

network = LinearNetwork(dim_input, dim_output)
optimizer = SGD(network.parameters(), 0.01)

training_loop(network, optimizer, train_data, dev_data, n_epochs=20)

Epoch 0 Loss 0.37560995818388165
Epoch 0, Dev Accuracy: 91.92%
Epoch 1 Loss 0.310088821724386
Epoch 1, Dev Accuracy: 92.08%
Epoch 2 Loss 0.29789616490333165
Epoch 2, Dev Accuracy: 91.65%
Epoch 3 Loss 0.28972922938106144
Epoch 3, Dev Accuracy: 92.46%
Epoch 4 Loss 0.28505334983295916
Epoch 4, Dev Accuracy: 92.58%
Epoch 5 Loss 0.2816775362680396
Epoch 5, Dev Accuracy: 92.24%
Epoch 6 Loss 0.27935619283753604
Epoch 6, Dev Accuracy: 92.39%
Epoch 7 Loss 0.2760014512394773
Epoch 7, Dev Accuracy: 92.59%
Epoch 8 Loss 0.2738697245527043
Epoch 8, Dev Accuracy: 92.60%
Epoch 9 Loss 0.2728910719273141
Epoch 9, Dev Accuracy: 92.57%
Epoch 10 Loss 0.2709716254248619
Epoch 10, Dev Accuracy: 92.58%
Epoch 11 Loss 0.26963090054481
Epoch 11, Dev Accuracy: 92.72%
Epoch 12 Loss 0.26773596340631656
Epoch 12, Dev Accuracy: 92.07%
Epoch 13 Loss 0.2671389395590722
Epoch 13, Dev Accuracy: 92.13%
Epoch 14 Loss 0.2667036146651501
Epoch 14, Dev Accuracy: 92.19%
Epoch 15 Loss 0.26539462946901726
Epoch 15, Dev Accuracy:

After you finished the linear network, you can move to a deep network!

In [ ]:
class DeepNetwork(ModuleList):
    def __init__(self, dim_input, dim_output, hidden_dim, n_layers, tanh=False):
        super().__init__()
        layer_dims = [dim_input] + [hidden_dim] * n_layers + [dim_output] 
        # for exemple  [784] + [64, 64] + [10] = [784, 64, 64, 10]
        print(layer_dims)
        for i in range(n_layers+1):
            W = Parameter(np.ndarray((layer_dims[i+1], layer_dims[i])), name="W"+str(i))
            b = Parameter(np.ndarray((layer_dims[i+1],)), name="b"+str(i))
            self.append(W)    
            self.append(b)
            if i < n_layers :
                self.append(TanH() if tanh else ReLU())
            # else :
            #     self.activations.append(Softmax())
        self.init_parameters()
            
        
    def init_parameters(self):
        # init parameters of the network (i.e W and b)
        for param in self.parameters():
            if isinstance(param, Parameter):
                if "W" in param.name:
                    glorot_init(param.value)
                elif "b" in param.name:
                    zero_init(param.value)

    def forward(self, x):
        # Assume that every two parameters are followed 
        # by an activation function (except the last pair).
        for i in range(0, len(self), 3):
            W = self[i]
            b = self[i+1]
            x = affine_transform()(W, b, x)
            if i + 2 < len(self):
                activation = self[i+2]
                x = activation(x)
        return x

In [ ]:
dim_input = 28*28
dim_output = 10

network = DeepNetwork(dim_input, dim_output, 128, 2,tanh=True)
optimizer = SGD(network.parameters(), 0.005)

training_loop(network, optimizer, train_data, dev_data, n_epochs=20)

[784, 128, 128, 10]


Epoch 0 Loss 0.2831445339313861
Epoch 0, Dev Accuracy: 95.49%
Epoch 1 Loss 0.1349101524773894
Epoch 1, Dev Accuracy: 96.65%
Epoch 2 Loss 0.0930918653458311
Epoch 2, Dev Accuracy: 97.05%
Epoch 3 Loss 0.07056370702509744
Epoch 3, Dev Accuracy: 97.15%
Epoch 4 Loss 0.054038355620496456
Epoch 4, Dev Accuracy: 97.42%
Epoch 5 Loss 0.0411039193892383
Epoch 5, Dev Accuracy: 97.23%
Epoch 6 Loss 0.03226454712437156
Epoch 6, Dev Accuracy: 97.43%
Epoch 7 Loss 0.02553800082553301
Epoch 7, Dev Accuracy: 97.69%
Epoch 8 Loss 0.01801570397479008
Epoch 8, Dev Accuracy: 97.73%
Epoch 9 Loss 0.013206966522790872
Epoch 9, Dev Accuracy: 97.68%
Epoch 10 Loss 0.009633769959014113
Epoch 10, Dev Accuracy: 97.74%
Epoch 11 Loss 0.007157462010867529
Epoch 11, Dev Accuracy: 97.77%
Epoch 12 Loss 0.0049626592780931626
Epoch 12, Dev Accuracy: 97.84%
Epoch 13 Loss 0.003767523974072972
Epoch 13, Dev Accuracy: 97.88%
Epoch 14 Loss 0.002934241795351017
Epoch 14, Dev Accuracy: 97.91%
Epoch 15 Loss 0.002355867891097761
Epoch 

## Better Optimizer
Implement the SGD with momentum, notice that you will need to store the cumulated gradient.


In [ ]:
class SGD:
    def __init__(self, params, lr=0.1, momentum=0.5):
        self.params = params
        self.lr = lr
        self.momentum = momentum
        self.velocities = [np.zeros_like(p.value) for p in self.params]
    def step(self):
        for i, p in enumerate(self.params):
            self.velocities[i] = self.momentum * self.velocities[i] - self.lr * p.grad
            p.value[:] = p.value + self.velocities[i]
    def zero_grad(self):
        for p in self.params:
            p.zero_grad()

In [ ]:
dim_input = 28*28
dim_output = 10

network = DeepNetwork(dim_input, dim_output, 128, 2,tanh=True)
optimizer = SGD(network.parameters(), 0.005)

training_loop(network, optimizer, train_data, dev_data, n_epochs=20)

[784, 128, 128, 10]
Epoch 0 Loss 0.25469738291169614
Epoch 0, Dev Accuracy: 95.60%
Epoch 1 Loss 0.11799272566302463
Epoch 1, Dev Accuracy: 96.96%
Epoch 2 Loss 0.08095845824358455
Epoch 2, Dev Accuracy: 97.16%
Epoch 3 Loss 0.060305005829908685
Epoch 3, Dev Accuracy: 96.74%
Epoch 4 Loss 0.04269164448416475
Epoch 4, Dev Accuracy: 96.76%
Epoch 5 Loss 0.03309211523605838
Epoch 5, Dev Accuracy: 97.52%
Epoch 6 Loss 0.02164025983200781
Epoch 6, Dev Accuracy: 97.34%
Epoch 7 Loss 0.015530513718558871
Epoch 7, Dev Accuracy: 97.51%
Epoch 8 Loss 0.010896108839154638
Epoch 8, Dev Accuracy: 97.68%
Epoch 9 Loss 0.005789169458003369
Epoch 9, Dev Accuracy: 97.81%
Epoch 10 Loss 0.002748290812503277
Epoch 10, Dev Accuracy: 98.00%
Epoch 11 Loss 0.0014415828710679101
Epoch 11, Dev Accuracy: 97.95%
Epoch 12 Loss 0.0009706740619321562
Epoch 12, Dev Accuracy: 97.95%
Epoch 13 Loss 0.00079955032895923
Epoch 13, Dev Accuracy: 98.00%
Epoch 14 Loss 0.0006942616789336818
Epoch 14, Dev Accuracy: 97.99%
Epoch 15 Loss 

## Bonus: Batch SGD
Propose a method to take into account batch of input

In [ ]:
class BatchSGD:
    def __init__(self, params, lr=0.1):
        self.params = params
        self.lr = lr
        
    def step(self, batch_size):
        for p in self.params:
            p.value[:] = p.value - self.lr * p.grad / batch_size
        
    def zero_grad(self):
        for p in self.params:
            p.zero_grad()

In [ ]:

def training_loop_batch(network, optimizer, train_data, dev_data, batch_size=32, n_epochs=10):
    for epoch in range(n_epochs):
        # Shuffle the data
        perm = np.random.permutation(train_data[0].shape[0])
        train_data_shuffled = train_data[0][perm], train_data[1][perm]
        total_loss = []

        for i in range(0, train_data_shuffled[0].shape[0], batch_size):
            # Extracting a mini-batch
            x_batch = train_data_shuffled[0][i:i+batch_size]
            y_batch = train_data_shuffled[1][i:i+batch_size]

            # Forward pass for a mini-batch
            x = ComputationGraphNode(x_batch)
            y_pred = network.forward(x)

            losses = [nll()(y_pred, y) for y in y_batch]
            batch_loss = sum(loss.value for loss in losses) / len(losses)
            total_loss.append(batch_loss)

            # Backward pass
            for loss in losses:
                loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Compute and print the average loss
        avg_loss = np.mean(total_loss)
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")
        
        dev_accuracy = evaluate_accuracy(network, dev_data)
        print(f"Epoch {epoch+1}, Dev Accuracy: {dev_accuracy * 100:.2f}%")


In [ ]:
class Operation(object):
    @staticmethod
    def forward(*args):
        raise NotImplementedError("It is an abstract method")

    @classmethod
    def __call__(self, *args):
        output_node = self.forward(*args)
        output_node.set_func(self)
        return output_node
        
    @staticmethod
    def backward(*args):
        pass

class Addition(Operation):
    @staticmethod
    def forward(x, y):
        output_array = x.vg_tensor.data + y.vg_tensor.data
        output_node = ComputationGraphNode(output_array)

        output_node.set_input_nodes(x, y)
        return output_node

    @staticmethod
    def backward(x, y, gradient):
        return (gradient, gradient)

class Power(Operation):
    @staticmethod
    def forward(x, power):
        np_x = x.vg_tensor.data
        np_power = power.vg_tensor.data
        output_array = np_x**np_power
        output_node = ComputationGraphNode(output_array)

        output_node.set_input_nodes(x, power)
        return output_node

    @staticmethod
    def backward(x, power, gradient):
        np_x = x.vg_tensor.data
        np_power = power.vg_tensor.data
        
        return  (np_power * np_x**(np_power-1) * gradient, np.log(np_x) * np_x**(np_power) * gradient)

class Multiplication(Operation):
    @staticmethod
    def forward(x, y):
        np_x = x.vg_tensor.data
        np_y = y.vg_tensor.data

        output_array = np_x * np_y 
        
        output_node = ComputationGraphNode(output_array)
        output_node.set_input_nodes(x, y)

        return output_node
        
    @staticmethod
    def backward(x, y, gradient):
        np_x = x.vg_tensor.data
        np_y = y.vg_tensor.data
        
        return (np_y * gradient, np_x * gradient) 
        
class Division(Operation):
    @staticmethod
    def forward(x, y):
        np_x = x.vg_tensor.data
        np_y = y.vg_tensor.data

        output_array = np_x / np_y 
        
        output_node = ComputationGraphNode(output_array)
        output_node.set_input_nodes(x, y)

        return output_node
        
    @staticmethod
    def backward(x, y, gradient):
        np_x = x.vg_tensor.data
        np_y = y.vg_tensor.data
        
        return (gradient/np_y, -(np_x/(np_y**2) * gradient)) 

class Selection(Operation):
    @staticmethod
    def forward(x, slice):
        np_x = x.vg_tensor.data

        output_array = np_x.__getitem__(slice)
        
        output_node = ComputationGraphNode(output_array)
        output_node.set_input_nodes(x)
        output_node.set_func_parameters(slice)

        return output_node
        
    @staticmethod
    def backward(x, slice, gradient):
        np_x = x.vg_tensor.data

        cgrad = np_x.copy()
        cgrad.fill(0)
        cgrad.__setitem__(slice, gradient)
        
        return cgrad
        
class VGTensor(object):
    def __init__(self, array_like):
        self.data = np.array(array_like, dtype=np.float64)
        self.grad = np.zeros_like(self.data, dtype=np.float64)
        
class ComputationGraphNode(object):
    
    def __init__(self, data, require_grad=False):
        # We initialise the value of the node and the grad
        self.value = np.array(data)
        self.vg_tensor = VGTensor(data)
        if require_grad:
            self.grad = np.zeros_like(self.value)
        else:
            self.grad = None
        self.require_grad = require_grad
        self.func = None
        self.input_nodes = None
        self.func_parameters = []
    
    def set_input_nodes(self, *nodes):
        self.input_nodes = list(nodes)

    def set_func_parameters(self, *func_parameters):
        self.func_parameters = list(func_parameters)
    
    def set_func(self, func):
        self.func = func

    def zero_grad(self):
        self.vg_tensor.grad.fill(0)

    def set_gradient(self, gradient):
        if self.vg_tensor.grad is None:
            self.vg_tensor.grad = gradient
        else:
            try:
                if gradient.shape != self.vg_tensor.grad.shape:
                    if gradient.shape == ():
                        gradient = np.full(self.vg_tensor.grad.shape, gradient)
                    else:
                        gradient = np.reshape(gradient, self.vg_tensor.grad.shape)
                self.vg_tensor.grad += gradient
            except ValueError as e:
                raise ValueError(f"Cannot reshape gradient of shape {gradient.shape} to {self.vg_tensor.grad.shape}") from e


    
    def backward(self, g=None):
        if g is None:
            g = self.vg_tensor.data.copy()
            g.fill(1.)
        else:
            self.set_gradient(g)
        if self.func is not None:
            grad_list = self.func.backward(*(self.input_nodes+ self.func_parameters + [g]))
            for input_node, ngrad in zip(self.input_nodes, grad_list):
                input_node.backward(ngrad)
    
    def show_graph(self):
        if self.func is not None:
            for x in self.input_nodes:
                x.show_graph()
    
    def __add__(self, y):
        if not isinstance(y, ComputationGraphNode):
            y = ComputationGraphNode(y)
        return Addition()(self, y)

    def __pow__(self, power):
        if not isinstance(power, ComputationGraphNode):
            power = ComputationGraphNode(power)
        return Power()(self, power)

    def __mul__(self, y):
        if not isinstance(y, ComputationGraphNode):
            y = ComputationGraphNode(y)
        return Multiplication()(self, y)
        
    def __truediv__(self, y):
        if not isinstance(y, ComputationGraphNode):
            y = ComputationGraphNode(y)
        return Division()(self, y)

    def __getitem__(self, slice):
        return Selection()(self, slice)

    def __str__(self):
        return self.vg_tensor.data.__str__()

    def __repr__(self):
        return self.vg_tensor.data.__str__()

class Parameter(ComputationGraphNode):
    def __init__(self, data, name="default"):
        super().__init__(data, require_grad=True)
        self.name  = name

    def backward(self, g):
        if g is not None:
            self.set_gradient(g)
    

In [ ]:
w1 = Parameter([0.1, 0.1])
b = Parameter([0.1])
x = ComputationGraphNode([0.5,-0.5])
y = 11

In [ ]:
epsilon = 2e-3

for i in range(100):
    o = w1[0] * x[0] + w1[1] * x[1] + b
    loss = (ComputationGraphNode([math.e])**o + (- y))**2 
    loss.backward()
    print(ComputationGraphNode([math.e])**o )
    w1.vg_tensor.data = w1.vg_tensor.data - epsilon * w1.vg_tensor.grad
    b.vg_tensor.data = b.vg_tensor.data - epsilon * b.vg_tensor.grad
    w1.zero_grad()
    b.zero_grad()

[1.10517092]
[1.15458608]
[1.20829638]
[1.26685385]
[1.33090403]
[1.40120558]
[1.47865446]
[1.56431441]
[1.65945488]
[1.76559901]
[1.88458382]
[2.01863608]
[2.17046732]
[2.34339175]
[2.54146998]
[2.76967794]
[3.03409263]
[3.34206746]
[3.70233212]
[4.12487816]
[4.62036009]
[5.19853876]
[5.86506792]
[6.61593388]
[7.42980587]
[8.26147487]
[9.04398733]
[9.70713099]
[10.2068694]
[10.54279287]
[10.74804037]
[10.86509911]
[10.92898661]
[10.96296747]
[10.9807852]
[10.99005661]
[10.99486156]
[10.99734653]
[10.99863026]
[10.99929307]
[10.99963519]
[10.99981175]
[10.99990286]
[10.99994987]
[10.99997413]
[10.99998665]
[10.99999311]
[10.99999645]
[10.99999817]
[10.99999905]
[10.99999951]
[10.99999975]
[10.99999987]
[10.99999993]
[10.99999997]
[10.99999998]
[10.99999999]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.]
[11.

/tmp/ipykernel_699811/747365483.py:45: RuntimeWarning: invalid value encountered in log
  return  (np_power * np_x**(np_power-1) * gradient, np.log(np_x) * np_x**(np_power) * gradient)
/tmp/ipykernel_699811/747365483.py:106: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cgrad.__setitem__(slice, gradient)
/tmp/ipykernel_699811/747365483.py:45: RuntimeWarning: divide by zero encountered in log
  return  (np_power * np_x**(np_power-1) * gradient, np.log(np_x) * np_x**(np_power) * gradient)
/tmp/ipykernel_699811/747365483.py:45: RuntimeWarning: invalid value encountered in multiply
  return  (np_power * np_x**(np_power-1) * gradient, np.log(np_x) * np_x**(np_power) * gradient)


In [ ]:
class Module(object):
    def __init__(self):
        pass
    def forward(self, *args, **kwargs):
        pass
    def __call__(self, *args, **kwargs):
        last_node = self.forward(*args, **kwargs)
        return last_node
        
    def parameters(self):
        parameters = []
        for name in dir(self):
            attribute = self.__getattribute__(name)
            if isinstance(attribute, 'Parameter'):
                parameters.append(attribute)
            elif isinstance(attribute, 'Module'):
                parameters += attribute.parameters()
        return parameters
            
class Linear(Module):
    def __init__(self, in_dim, out_dim):
        super().__init__() 
        self.W = Parameter(np.array((in_dim, out_dim)), name='W')
        self.b = Parameter(np.array((out_dim)), name='b')
    def forward(self, x):
        return np.dot(self.W.vg_tensor.data, x) + self.b.vg_tensor.data